In [1]:
import numpy as np
import string 
import scipy.io
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
import itertools
from sklearn import svm
import statsmodels.api as sm # import statsmodels 
from scipy import signal


class Scan(object):
    def __init__(self,activations,timestamp, step,prev_words=None,next_words=None,all_words=None,all_pos=None,all_speak_features=None,current_translated_words=None):
        self.activations = activations
        self.timestamp = timestamp
        self.prev_words = prev_words
        self.next_words = next_words
        self.step = step
        self.all_words = all_words
        self.current_translated_words = current_translated_words
        self.all_pos = all_pos
        self.all_speak_features = all_speak_features
        self.brain3d = None
        
def eval(dists,e_dists):
    nn_index = np.argmin(dists,axis=1)
    accuracy_on_test = np.mean(nn_index == np.argmax(np.eye(dists.shape[0]),axis=1))


    b_acc = []
    e_b_acc = []
    for i,j in itertools.combinations(np.arange(dists.shape[0]), 2):
        right_match = dists[i,i] + dists[j,j]
        wrong_match = dists[i,j] + dists[j,i]
        b_acc.append(right_match < wrong_match)

        e_right_match = e_dists[i,i] + e_dists[j,j]
        e_wrong_match = e_dists[i,j] + e_dists[j,i]
        e_b_acc.append(e_right_match < e_wrong_match)

    #print("binary accuracy: ", np.mean(b_acc)," ", np.mean(e_b_acc))
    return np.mean(b_acc),np.mean(e_b_acc),b_acc,e_b_acc

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
story_features = scipy.io.loadmat('../data/story_features.mat') 

In [3]:
speach_feature_id = 1
motion_feature_id = 2
emotion_feature_id = 3
verbs_feature_id = 4
characters_feature_id = 5
visual_wordlength_feature_id = 6
Word_Num_feature_id = 7
part_of_speaches_feature_id = 8
Dependency_role_feature_id = 9

In [4]:
part_of_speaches = story_features['features'][0][part_of_speaches_feature_id][1][0]

In [5]:
story_features['features'][0][speach_feature_id]
story_features['features'][0][characters_feature_id][2].shape

(5176, 10)

In [6]:
subject_id = 1
fMRI_preprocessing_mode = "detrend_filter_std" 

block_pos = {}

for block_id in [1,2,3,4]:
    block_pos[block_id] = np.load("subject_"+str(subject_id)+"_block_"+str(block_id)+"_pos.npy")

block_scans = np.load("../processed_data/"+"subject_"+str(subject_id)+"/"+fMRI_preprocessing_mode+"subject_"+str(subject_id)+"_scan_objects.npy")

In [7]:
print(len(block_scans.item()[1]))
print(len(block_scans.item()[2]))
print(len(block_scans.item()[3]))
print(len(block_scans.item()[4]))

326
338
265
366


In [19]:
embeddings_0 = np.load("../embeddings/subject_"+str(1)+"_lstm_"+str(0)+"_emb_objects.npy")
embeddings_1 = np.load("../embeddings/subject_"+str(1)+"_lstm_"+str(1)+"_emb_objects.npy")

LSTM_embeddings_H_0 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_H_1 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_H_01 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_M_0 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_M_1 = {1:[],2:[],3:[],4:[]}
LSTM_embeddings_M_01 = {1:[],2:[],3:[],4:[]}

for block_id in [1,2,3,4]:    
    print("block id",block_id)
    layer_id = 0
    lstm_h_0 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_hidden_"+str(layer_id)+".npy")
    lstm_c_0 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_memory_"+str(layer_id)+".npy")
    layer_id = 1
    lstm_h_1 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_hidden_"+str(layer_id)+".npy")
    lstm_c_1 = np.load("../../lm1b/text_input_full_state/block_"+str(block_id)+"/lstm_memory_"+str(layer_id)+".npy")
  
    for step in np.arange(len(block_texts[block_id])):
        LSTM_embeddings_H_0[block_id].append(lstm_h_0.item()[step][0])
        LSTM_embeddings_H_1[block_id].append(lstm_h_1.item()[step][0])
        LSTM_embeddings_H_01[block_id].append(np.concatenate([lstm_h_0.item()[step][0],lstm_h_1.item()[step][0]]))
        
        LSTM_embeddings_M_0[block_id].append(lstm_c_0.item()[step][0])
        LSTM_embeddings_M_1[block_id].append(lstm_c_1.item()[step][0])
        LSTM_embeddings_M_01[block_id].append(np.concatenate([lstm_c_0.item()[step][0],lstm_c_1.item()[step][0]]))

block id 1


NameError: name 'block_texts' is not defined

In [9]:
block_scans.item()[block_id][0]

In [10]:
detrended_block_scans = {1:[],2:[],3:[],4:[]}
raw_block_scans = {1:[],2:[],3:[],4:[]}

for block_id in [1,2,3,4]:
        for i in np.arange(len(block_scans.item()[block_id])):
            raw_block_scans[block_id].append(block_scans.item()[block_id][i].activations[0])
                           
        detrended_block_scans[block_id] = signal.detrend(raw_block_scans[block_id],type="constant")

In [11]:
non_fixed_indexes = np.where(np.min(train_brain_activations,axis=0) != np.max(train_brain_activations,axis=0))
non_fixed_indexes

NameError: name 'train_brain_activations' is not defined

In [17]:
def prepare_data(train_block_ids,test_block_ids):
    train_features = {'position':[],'pos_tag':[],'speak_tag':[],'lstm_1':[],'lstm_0':[],'lstm_prev_1':[],'lstm_prev_0':[]}
    train_brain_activations = []
    for train_block_id in train_block_ids:
        for i in np.arange(len(block_scans.item()[train_block_id])):
            scan = block_scans.item()[train_block_id][i]
            if (scan.step - 4) in embeddings_1.item()[train_block_id].keys():
                train_features['position'].append(scan.step)
                train_features['pos_tag'].append(np.sum(scan.all_pos,axis=0))
                train_features['speak_tag'].append(np.eye(2)[min([1,np.sum(np.asarray(scan.all_speak_features)[:,0])])])
                train_features['lstm_1'].append(np.mean(embeddings_1.item()[train_block_id][scan.step],axis=0))
                train_features['lstm_0'].append(np.mean(embeddings_0.item()[train_block_id][scan.step],axis=0))
                train_features['lstm_prev_1'].append(np.mean(embeddings_1.item()[train_block_id][scan.step-4],axis=0))
                train_features['lstm_prev_0'].append(np.mean(embeddings_0.item()[train_block_id][scan.step-4],axis=0))
                train_brain_activations.append(scan.activations[0])
        #print(scan.step)

    test_features = {'position':[],'pos_tag':[],'speak_tag':[],'lstm_1':[],'lstm_0':[],'lstm_prev_1':[],'lstm_prev_0':[]}
    test_brain_activations = []
    for test_block_id in test_block_ids:
        for scan in block_scans.item()[test_block_id]:
            if (scan.step - 4) in embeddings_1.item()[test_block_id].keys():
                test_features['position'].append(scan.step)
                test_features['pos_tag'].append(np.sum(scan.all_pos,axis=0))
                test_features['speak_tag'].append(np.eye(2)[min([1,np.sum(np.asarray(scan.all_speak_features)[:,0])])])
                test_features['lstm_1'].append(np.mean(embeddings_1.item()[test_block_id][scan.step],axis=0))
                test_features['lstm_0'].append(np.mean(embeddings_0.item()[test_block_id][scan.step],axis=0))
                test_features['lstm_prev_1'].append(np.mean(embeddings_1.item()[test_block_id][scan.step-4],axis=0))
                test_features['lstm_prev_0'].append(np.mean(embeddings_0.item()[test_block_id][scan.step-4],axis=0))
                test_brain_activations.append(scan.activations[0])
            #print(scan.step)
    return train_features,train_brain_activations,test_features,test_brain_activations


def train_model(X,y):
    #X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
    # Note the difference in argument order
    model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
    
    return model


def MRR(distances):
    prec_at_corrects = []
    ranks = []
    sorted_indexes = np.argsort(distances,axis=1)
    for i in np.arange(len(distances)):
        #print(i)
        correct_at = np.where(sorted_indexes[i] == i)[0] + 1
        #print("Reciprocal Rank",correct_at)
        prec_at_correct = 1.0/correct_at
        #print("precision at ",correct_at,": ",prec_at_correct)
        prec_at_corrects.append(prec_at_correct)
        ranks.append(correct_at)
    
    print("MRR: ",np.mean(prec_at_corrects)," ",np.mean(ranks))
    return np.mean(ranks), np.mean(prec_at_corrects), ranks,prec_at_corrects

def test_model(model,X_t,y_t):
    #X_t = sm.add_constant(X_t) ## let's add an intercept (beta_0) to our model
    pred_t = model.predict(X_t)
    
    
    argmax_accuracy = np.mean(np.argmax(pred_t, axis=1) == np.argmax(y_t,axis=1))
    print("check argmax dim:",np.argmax(pred_t, axis=1).shape)
    print("argmax accuracy:",argmax_accuracy)
    cosine_dists = distance.cdist(pred_t,y_t,'cosine')
    euc_dists =  distance.cdist(pred_t,y_t,'euclidean')
    
    print("cosine dist >>")
    mean_ranks_c = MRR(cosine_dists)
    
    print("euc_dists dist >>")
    mean_ranks_e = MRR(euc_dists)
    
    print("binary accuracy >>")
    c_acc, e_acc, _,_ = eval(cosine_dists,euc_dists)
    print(c_acc,e_acc)
    
    return c_acc, e_acc, argmax_accuracy

    


In [57]:

train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)

train_brain_activations = np.asarray(train_brain_activations)
non_fixed_indexes = np.where(np.min(train_brain_activations,axis=0) != np.max(train_brain_activations,axis=0))
fixed_indexes = np.where(np.min(train_brain_activations,axis=0) == np.max(train_brain_activations,axis=0))

train_brain_activations[:,non_fixed_indexes] = (train_brain_activations[:,non_fixed_indexes] - np.min(train_brain_activations[:,non_fixed_indexes],axis=0)) / (np.max(train_brain_activations[:,non_fixed_indexes],axis=0) - np.min(train_brain_activations[:,non_fixed_indexes],axis=0))
train_brain_activations[:,fixed_indexes] = 0

test_brain_activations = np.asarray(test_brain_activations)
non_fixed_indexes = np.where(np.min(test_brain_activations,axis=0) != np.max(test_brain_activations,axis=0))
fixed_indexes = np.where(np.min(test_brain_activations,axis=0) == np.max(test_brain_activations,axis=0))

test_brain_activations[:,non_fixed_indexes] = (test_brain_activations[:,non_fixed_indexes] - np.min(test_brain_activations[:,non_fixed_indexes],axis=0)) / (np.max(test_brain_activations[:,non_fixed_indexes],axis=0) - np.min(test_brain_activations[:,non_fixed_indexes],axis=0))
test_brain_activations[:,fixed_indexes] = 0
                                                                                                                          
X = train_features['position'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['position'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)


train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['position'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['position'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)



train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['position'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['position'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)



train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['position'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['position'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)



check argmax dim: (365,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.017749266454767033   183.0
euc_dists dist >>
MRR:  0.01983800414148042   183.5068493150685
binary accuracy >>
0.5004817100707512 0.4912690049676351
check argmax dim: (264,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.023314611787025357   132.5
euc_dists dist >>
MRR:  0.02331875342024551   132.47348484848484
binary accuracy >>
0.45019587510081804 0.5302454199792603
check argmax dim: (337,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.0189874823175414   169.0
euc_dists dist >>
MRR:  0.01897547379942712   169.01186943620178
binary accuracy >>
0.41961636286562104 0.48270806839056096
check argmax dim: (325,)
argmax accuracy: 0.0
cosine dist >>
MRR:  0.01957716467635644   163.0
euc_dists dist >>
MRR:  0.0198405839435279   163.0430769230769
binary accuracy >>
0.42894586894586895 0.5178917378917379


(0.42894586894586895, 0.5178917378917379, 0.0)

In [45]:
train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['pos_tag'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['pos_tag'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['pos_tag'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['pos_tag'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['pos_tag'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['pos_tag'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_features['pos_tag'] ## X usually means our input variables (or independent variables)
y = train_brain_activations ## Y usually means our output/dependent variable
X_t = test_features['pos_tag'] ## X usually means our input variables (or independent variables)
y_t = test_brain_activations ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)


check argmax dim: (365,)
argmax accuracy: 0.03287671232876712
cosine dist >>
MRR:  0.01983853200104721   176.5095890410959
euc_dists dist >>
MRR:  0.017840442856963182   182.85753424657534
binary accuracy >>
0.5180641276531688 0.5175673641427067
check argmax dim: (264,)
argmax accuracy: 0.026515151515151516
cosine dist >>
MRR:  0.032922870749266336   128.28030303030303
euc_dists dist >>
MRR:  0.021976280313488705   131.47727272727272
binary accuracy >>
0.5156123977416753 0.5220071436801474
check argmax dim: (337,)
argmax accuracy: 0.04747774480712166
cosine dist >>
MRR:  0.01834661662300858   168.56379821958458
euc_dists dist >>
MRR:  0.016415608494820412   167.93175074183975
binary accuracy >>
0.5054048325561679 0.5073477462201498
check argmax dim: (325,)
argmax accuracy: 0.03076923076923077
cosine dist >>
MRR:  0.017680779114457392   164.43692307692308
euc_dists dist >>
MRR:  0.015738994964689078   164.92923076923077
binary accuracy >>
0.4914909781576448 0.49240265906932573


(0.4914909781576448, 0.49240265906932573, 0.03076923076923077)

In [18]:
train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)
    

train_brain_activations = train_brain_activations - np.mean(train_brain_activations,axis=0)
test_brain_activations = test_brain_activations - np.mean(test_brain_activations,axis=0)

#non_fixed_indexes = np.where(np.min(train_brain_activations,axis=0) != np.max(train_brain_activations,axis=0))
#fixed_indexes = np.where(np.min(train_brain_activations,axis=0) == np.max(train_brain_activations,axis=0))

#train_brain_activations[:,non_fixed_indexes] = (train_brain_activations[:,non_fixed_indexes] - np.min(train_brain_activations[:,non_fixed_indexes],axis=0)) / (np.max(train_brain_activations[:,non_fixed_indexes],axis=0) - np.min(train_brain_activations[:,non_fixed_indexes],axis=0))
#train_brain_activations[:,fixed_indexes] = 0

#test_brain_activations = np.asarray(test_brain_activations)
#non_fixed_indexes = np.where(np.min(test_brain_activations,axis=0) != np.max(test_brain_activations,axis=0))
#fixed_indexes = np.where(np.min(test_brain_activations,axis=0) == np.max(test_brain_activations,axis=0))

#test_brain_activations[:,non_fixed_indexes] = (test_brain_activations[:,non_fixed_indexes] - np.min(test_brain_activations[:,non_fixed_indexes],axis=0)) / (np.max(test_brain_activations[:,non_fixed_indexes],axis=0) - np.min(test_brain_activations[:,non_fixed_indexes],axis=0))
#test_brain_activations[:,fixed_indexes] = 0


print(np.asarray(train_features['lstm_0']).shape)
X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)


train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)

print(train_features['lstm_0'].shape)
X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

(926,)


AxisError: axis 1 is out of bounds for array of dimension 1

In [47]:
delay = 5

train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

model = train_model(X[delay:],y[:len(y)-delay])
test_model(model,X_t[delay:],y_t[:len(y_t)-delay])


train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X[delay:],y[:len(y)-delay])
test_model(model,X_t[delay:],y_t[:len(y_t)-delay])

train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X[delay:],y[:len(y)-delay])
test_model(model,X_t[delay:],y_t[:len(y_t)-delay])

train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_brain_activations) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_brain_activations) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X[delay:],y[:len(y)-delay])
test_model(model,X_t[delay:],y_t[:len(y_t)-delay])

check argmax dim: (360,)
argmax accuracy: 0.05555555555555555
cosine dist >>
MRR:  0.018087302063027173   178.12777777777777
euc_dists dist >>
MRR:  0.015611809681056377   179.57222222222222
binary accuracy >>
0.5114360878984835 0.5109408851748685
(1027, 2048) (1027, 37913)
(264, 2048) (264, 37913)
check argmax dim: (259,)
argmax accuracy: 0.05405405405405406
cosine dist >>
MRR:  0.019447474646161717   123.96911196911196
euc_dists dist >>
MRR:  0.018246937112238366   126.82239382239382
binary accuracy >>
0.5292867618449014 0.5331477657059053
(954, 2048) (954, 37913)
(337, 2048) (337, 37913)
check argmax dim: (332,)
argmax accuracy: 0.060240963855421686
cosine dist >>
MRR:  0.028220856931570056   162.0843373493976
euc_dists dist >>
MRR:  0.019627307770516616   160.19277108433735
binary accuracy >>
0.5242237833509263 0.5215120299930841
(966, 2048) (966, 37913)
(325, 2048) (325, 37913)
check argmax dim: (320,)
argmax accuracy: 0.059375
cosine dist >>
MRR:  0.024801250260915376   151.45
eu

(0.5352076802507837, 0.5375783699059561, 0.059375)

In [48]:
len(X[1])

2048

In [49]:
true_mu = 361 / 2.0

onesample_results = scipy.stats.ttest_1samp(ranks, true_mu)
print("ranks: ", onesample_results)

NameError: name 'ranks' is not defined

In [ ]:
train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)


train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

In [ ]:
train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_brain_activations#np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = test_brain_activations #np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)


train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_brain_activations#np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = test_brain_activations#np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_brain_activations# np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = test_brain_activations #np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = train_brain_activations#np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['speak_tag']) ## Y usually means our output/dependent variable
X_t = test_brain_activations #np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['speak_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

In [ ]:
train_block_ids = [1,2,3]
test_block_ids = [4]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['pos_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['pos_tag']) ## Y usually means our output/dependent variable

model = train_model(X,y)
test_model(model,X_t,y_t)


train_block_ids = [1,2,4]
test_block_ids = [3]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['pos_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['pos_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [1,3,4]
test_block_ids = [2]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['pos_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['pos_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)

train_block_ids = [2,3,4]
test_block_ids = [1]
train_features,train_brain_activations,test_features,test_brain_activations = \
                                                prepare_data(train_block_ids,test_block_ids)


X = np.concatenate([train_features['lstm_0'],train_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y = np.asarray(train_features['pos_tag']) ## Y usually means our output/dependent variable
X_t = np.concatenate([test_features['lstm_0'],test_features['lstm_1']],axis=1) ## X usually means our input variables (or independent variables)
y_t = np.asarray(test_features['pos_tag']) ## Y usually means our output/dependent variable

print(X.shape,y.shape)
print(X_t.shape,y_t.shape)
model = train_model(X,y)
test_model(model,X_t,y_t)